In [17]:
#This following Method will read a bunch of circuits and calculate the circuit depth of each of them lols.
from qiskit import *
import os
from qiskit.circuit.random import random_circuit
from qiskit.providers.fake_provider import FakeSherbrooke
from mqt.bench import *
from qiskit import *
from qiskit.circuit.library import *
from qiskit.quantum_info import *
from qiskit.algorithms import *
from qiskit import *
from qiskit.providers.fake_provider import FakeSherbrooke
from qiskit.circuit.random import random_circuit
import numpy as np

backend = FakeSherbrooke()

In [5]:
#This method takes all the circuits built by MQT and makes them into a list
def file_reader(file_path):
    circuits = []
    directory = file_path
    for circuit in os.listdir(directory):
        circuit_path = f"{file_path}/{circuit}"
        print(circuit_path)
        qc = QuantumCircuit.from_qasm_file(circuit_path)
        circuits.append(qc)
    return circuits

In [22]:
MQT_circuits = file_reader("/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22")


/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_6.qasm
/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_7.qasm
/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_11.qasm
/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_10.qasm
/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_2.qasm
/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_13.qasm
/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_12.qasm
/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_3.qasm
/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_8.qasm
/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_4.qasm
/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_14.qasm
/Users/noelnegron/Desktop/MQTBench_2023-07-13-16-50-22/random_indep_qiskit_5.qasm
/Users/noel

In [11]:
def benchmark_circuit_depth(circuits):
    backend = FakeSherbrooke()
    Map = {}
    counter = 1
    for i in range(9):
        depths = []
        depths.append(transpile(circuits[i], optimization_level=1, backend = backend).depth())
        depths.append(transpile(circuits[i], optimization_level=2, backend = backend).depth())
        depths.append(transpile(circuits[i], optimization_level=3, backend = backend).depth())
        Map[i] = depths
        print(counter)
        counter += 1
    return Map

In [28]:
def num_single_and_multi_qubit_gates(circuit):
    Map = {'single' : 0, "multi" : 0}
    for gate in circuit.data:
        if len(gate[1]) == 1:
            Map['single'] = Map['single'] + 1
        else:
            Map['multi'] = Map['multi'] + 1
    return Map

In [32]:
import matplotlib.pyplot as plt
def single_multi_ratio_benchmarking(circuits):
    
    Map = {"Single" : 0, "Multi": 0}
    
    #These list will store the ratios (single // Multi ) of each circuit
    level1_list = []
    level2_list = []
    level3_list = []    
    return_list = []
    
    counter = 0
    #Step 1: iterate through all circuits
    for circuit in circuits:
        #Transpile the circuit on all three levels
        qc1 = transpile(circuit, optimization_level= 1, seed_transpiler= 42, backend=backend)
        qc2 = transpile(circuit, optimization_level= 2, seed_transpiler= 42, backend=backend)
        qc3 = transpile(circuit, optimization_level= 3, seed_transpiler= 42, backend=backend)
        print(f"circuit index: {counter} :completed")
        counter = counter + 1
    
        
        #Step 2: Transpile the circuit with level 1, 2 and 3 and collect the data. The return type of this object is a Map with key value pairs 
        # 'single' -- > Number of Single-Qubit Gates           'multi' --> Number of multi-qubit gates
        #Update Map Accordingly
        level1_data = num_single_and_multi_qubit_gates(qc1)
        level2_data = num_single_and_multi_qubit_gates(qc2)
        level3_data = num_single_and_multi_qubit_gates(qc3)
        
        Map["Single"] = level1_data['single'] + level2_data['single'] + level3_data['single']
        Map["Multi"] = level1_data['multi'] + level2_data['multi'] + level3_data['multi']
        
        #Step 3: Prepare ratio and list that contains all the data
        if level1_data['multi'] > 0:
            level1_ratio = level1_data['single'] / level1_data['multi']
            level1_list.append(level1_ratio)
        else:
            level1_list.append(level1_data['single'])
            
        if level2_data['multi'] > 0:
            level2_ratio = level2_data['single'] / level2_data['multi']
            level2_list.append(level2_ratio)
        else:
            level2_list.append(level1_data['single'])
            
        if level3_data['multi'] > 0:
            level3_ratio = level3_data['single'] / level3_data['multi']
            level3_list.append(level3_ratio)
        else:
            level3_list.append(level1_data['single'])
    
        
    return_list.append(level1_list), return_list.append(level2_list), return_list.append(level3_list)
    number_of_circuits = []
    
    for i in range(counter):
        circuit_number = i + 1
        number_of_circuits.append(circuit_number)
    
    #Line 1 --> Optimization Level 1.. etc
    plt.scatter(number_of_circuits, level1_list , label = "Optimization Level 1")
    plt.scatter(number_of_circuits, level2_list, label = "Optimization Level 2")
    plt.scatter(number_of_circuits, level3_list, label = "Optimization Level 3")
    plt.legend()
    plt.show()  
    
    return return_list

In [33]:
#This obtains Circuit Depth:
data = single_multi_ratio_benchmarking(MQT_circuits)

/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, sign

circuit index: 0 :completed


/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, sign

circuit index: 1 :completed


/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, sign

...

/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, sign

circuit index: 3 :completed


/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


circuit index: 4 :completed


/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, sign